<a href="https://colab.research.google.com/github/tj191073-droid/tj191073/blob/main/fishnet3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://huggingface.co/datasets/imageomics/fish-vista

Cloning into 'fish-vista'...
remote: Enumerating objects: 80157, done.
remote: Total 80157 (delta 0), reused 0 (delta 0), pack-reused 80157 (from 1)
Receiving objects: 100% (80157/80157), 212.04 MiB | 16.23 MiB/s, done.
Resolving deltas: 100% (104/104), done.
Updating files: 100% (75833/75833), done.
Filtering content: 100% (75699/75699), 11.04 GiB | 19.52 MiB/s, done.


In [ ]:
%cd fish-vista
!git lfs install
!git lfs pull

/content/fish-vista
Updated git hooks.
Git LFS initialized.


In [ ]:
import os
print(os.path.exists("/content/fish-vista/classification_train.csv"))

True


In [ ]:
import pandas as pd
import os

# 1. 加载原始数据
df = pd.read_csv("/content/fish-vista/classification_train.csv", low_memory=False)

# 2. 统计每个鱼科下的图像数量（按物种聚合）
grouped = df.groupby(["family", "standardized_species"]).size().reset_index(name='count')

# 3. 筛选出图像数量 >= 30 的物种
grouped = grouped[grouped['count'] >= 30]

# 4. 每个鱼科最多随机选 2 个物种（鱼类），最多保留 5 个鱼科
selected_families = grouped.groupby("family")['count'].sum().sort_values(ascending=False).head(5).index.tolist()
filtered = grouped[grouped['family'].isin(selected_families)]

# 5. 每个鱼科最多选 2 个物种
selected_species = (
    filtered.groupby("family")
    .apply(lambda x: x.sample(n=min(len(x), 2), random_state=42))
    .reset_index(drop=True)
)

# 6. 从原始 df 中筛选图像记录
sub_df = df[df['standardized_species'].isin(selected_species['standardized_species'])]

# 7. 每个物种最多取 30 张
subset = sub_df.groupby('standardized_species', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 30), random_state=42)
).reset_index(drop=True)

# 8. 构建完整图像路径
def resolve_path(file_name):
    try:
        chunk = file_name.split('/')[1].split('_')[1]
        return os.path.join("/content/fish-vista/Images", f"chunk_{chunk}", os.path.basename(file_name))
    except:
        return None

subset['image_path'] = subset['file_name'].apply(resolve_path)
subset = subset[subset['image_path'].notnull()]
subset['image_exists'] = subset['image_path'].apply(lambda x: os.path.exists(x))
subset = subset[subset['image_exists']].reset_index(drop=True)

# 9. 展示摘要
print(f"✅ 最终子集包含 {subset['family'].nunique()} 个鱼科，{subset['standardized_species'].nunique()} 个物种，{len(subset)} 张图像")
print(subset[['family', 'standardized_species']].value_counts())

✅ 最终子集包含 8 个鱼科，10 个物种，299 张图像
family         standardized_species
Centrarchidae  lepomis megalotis       30
               lepomis miniatus        30
Cottidae       cottus perplexus        30
Cyprinidae     notropis rubellus       30
Esocidae       esox americanus         30
Ictaluridae    noturus exilis          30
Esocidae       esox lucius             30
Cottidae       cottus carolinae        29
Cyprinidae     notropis telescopus     28
Ictaluridae    noturus eleutherus      28
ictaluridae    noturus eleutherus       2
cottidae       cottus carolinae         1
cyprinidae     notropis telescopus      1
Name: count, dtype: int64


/tmp/ipython-input-4-3213855260.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(len(x), 2), random_state=42))
/tmp/ipython-input-4-3213855260.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subset = sub_df.groupby('standardized_species', group_keys=False).apply(


In [ ]:
import shutil
from sklearn.model_selection import train_test_split

# 指定输出目录
base_dir = "/content/fish_dataset_mini"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

# 清空旧目录
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

# 创建 train/val 目录
for d in [train_dir, val_dir]:
    os.makedirs(d, exist_ok=True)

# 按类别划分图像
for cls in subset["standardized_species"].unique():
    cls_df = subset[subset["standardized_species"] == cls]
    image_paths = cls_df["image_path"].tolist()
    image_paths = [p for p in image_paths if isinstance(p, str)]  # 防止 None

    train_imgs, val_imgs = train_test_split(image_paths, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    for src in train_imgs:
        shutil.copy(src, os.path.join(train_dir, cls, os.path.basename(src)))
    for src in val_imgs:
        shutil.copy(src, os.path.join(val_dir, cls, os.path.basename(src)))

print("✅ 图像划分完成（train/val）")

✅ 图像划分完成（train/val）


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# ========= 配置路径 =========
data_dir = '/content/fish_dataset_mini'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# ========= 图像预处理 =========
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

class_names = train_dataset.classes
num_classes = len(class_names)

# ========= 定义 RAL（Robust Asymmetric Loss） =========
class RALLoss(nn.Module):
    def __init__(self, gamma_pos=0.5, gamma_neg=1.5):
        super(RALLoss, self).__init__()
        self.gamma_pos = gamma_pos
        self.gamma_neg = gamma_neg

    def forward(self, logits, targets):
        probs = torch.softmax(logits, dim=1)
        targets_one_hot = torch.zeros_like(probs).scatter_(1, targets.unsqueeze(1), 1)
        pt = torch.sum(probs * targets_one_hot, dim=1)

        focal_weight = (1 - pt) ** self.gamma_pos
        loss = -focal_weight * torch.log(pt + 1e-8)
        return loss.mean()

# ========= 初始化模型 =========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=True)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model = model.to(device)

# ========= 设置损失函数与优化器 =========
criterion = RALLoss(gamma_pos=0.5, gamma_neg=1.5)
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# ========= 开始训练 =========
best_val_acc = 0.0
for epoch in range(1, 21):
    model.train()
    correct, total = 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # 验证过程
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total

    print(f"Epoch {epoch}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")

    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model_ral.pth")
        print("✅ 模型已保存：best_model_ral.pth")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 88.3MB/s]


Epoch 1: Train Acc = 0.5607, Val Acc = 0.6833
✅ 模型已保存：best_model_ral.pth
Epoch 2: Train Acc = 0.9289, Val Acc = 0.8167
✅ 模型已保存：best_model_ral.pth
Epoch 3: Train Acc = 0.9874, Val Acc = 0.8167
Epoch 4: Train Acc = 0.9958, Val Acc = 0.8167
Epoch 5: Train Acc = 0.9916, Val Acc = 0.8167
Epoch 6: Train Acc = 0.9916, Val Acc = 0.8167
Epoch 7: Train Acc = 0.9916, Val Acc = 0.8333
✅ 模型已保存：best_model_ral.pth
Epoch 8: Train Acc = 0.9874, Val Acc = 0.8333
Epoch 9: Train Acc = 0.9791, Val Acc = 0.7833
Epoch 10: Train Acc = 0.9874, Val Acc = 0.7833
Epoch 11: Train Acc = 0.9791, Val Acc = 0.7000
Epoch 12: Train Acc = 0.9791, Val Acc = 0.8000
Epoch 13: Train Acc = 0.9874, Val Acc = 0.7833
Epoch 14: Train Acc = 0.9958, Val Acc = 0.8833
✅ 模型已保存：best_model_ral.pth
Epoch 15: Train Acc = 0.9958, Val Acc = 0.8833
Epoch 16: Train Acc = 1.0000, Val Acc = 0.8667
Epoch 17: Train Acc = 1.0000, Val Acc = 0.8500
Epoch 18: Train Acc = 1.0000, Val Acc = 0.8333
Epoch 19: Train Acc = 1.0000, Val Acc = 0.8500
Epoch 20

In [ ]:
from google.colab import files
uploaded = files.upload()

# 获取上传文件的路径
import os
image_path = list(uploaded.keys())[0]
print(f"📸 上传图片路径：{image_path}")

Saving INHS_FISH_102455.jpg to INHS_FISH_102455.jpg
📸 上传图片路径：INHS_FISH_102455.jpg


In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import torch.nn as nn
from torchvision import models

# 类别信息（与你训练集一致）
class_names = sorted(os.listdir('/content/fish_dataset_mini/train'))
num_classes = len(class_names)

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型
model = models.mobilenet_v2(pretrained=False)
model.classifier[1] = nn.Linear(model.last_channel, num_classes)
model.load_state_dict(torch.load("best_model_ral.pth", map_location=device))
model.to(device)
model.eval()

# 图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 加载图片
img = Image.open(image_path).convert('RGB')
img_tensor = transform(img).unsqueeze(0).to(device)

# 预测
with torch.no_grad():
    outputs = model(img_tensor)
    _, predicted = torch.max(outputs, 1)
    pred_class = class_names[predicted.item()]

print(f"🔍 预测结果：{pred_class}")

🔍 预测结果：lepomis megalotis
